In [ ]:
import igraph 
import csv
import unicodedata

In [ ]:
def PrintGraph(resultGraph, toPrint,result, Overlap, iteration, printEvery):
    index = 1
    while index<=iteration:
        x = result[len(result)-index]
        
        for edge in x:
            #same source and target of the edge mean that me have vertex copy in this step
            if(edge[0]==edge[1]):
                continue
            resultGraph.add_edges([edge])
            
        comp = [[] for i in range(0, len(resultGraph.components()))]
        counter = 0
        for component in resultGraph.components():
            #skip solo community
            if(len(component)==1 and component[0] >= toPrint.vcount()):
                continue
            
            toAdd = []
            #find original number of the vertex from his copies
            for vertex in component:
                if(vertex<toPrint.vcount()):
                    toAdd.append(vertex)
                else:
                    while(Overlap[vertex]!=-1):
                        vertex=Overlap[vertex]
                    toAdd.append(vertex)
            comp[counter] = toAdd
            counter+=1
        
        #prepare graph to print
        for edge in x:
            source = edge[0]
            target = edge[1]
            if(source == target):
                continue
            if(source>=toPrint.vcount()): 
                while(Overlap[source]!=-1):
                    source=Overlap[source]
            if(target>=toPrint.vcount()):
                while(Overlap[target]!=-1):
                    target=Overlap[target]
            toPrint.add_edges([(source,target)])
        
        toPrint.simplify(multiple=True, loops=True, combine_edges=None)
        
        #assign communities to all vertices
        counter = 0
        for vertex in toPrint.vs:
            vertex["communities"] = []
        for c in comp:
            for vertex in c:
                if counter not in toPrint.vs[vertex]["communities"]:
                    toPrint.vs[vertex]["communities"].append(counter)
            counter+=1
            
        #print functions
        if printEvery == True or index == iteration:
            print("Result graph communities:")
            print(resultGraph.components())
            plot_full(toPrint, str(index)+"Iteration.svg")
        
        
        index+=1